# Neural Machine Translation

**Reference:** Sutskever, Ilya, Oriol Vinyals, and Quoc V. Le. "Sequence to sequence learning with neural networks." In Advances in neural information processing systems, pp. 3104-3112. 2014. ([Paper](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks), [Sample code](https://github.com/tensorflow/nmt))

In [1]:
using Knet, Test, Base.Iterators, IterTools, Random # , LinearAlgebra, StatsBase
using AutoGrad: @gcheck  # to check gradients, use with Float64
Knet.atype() = KnetArray{Float32}  # determines what Knet.param() uses.
macro size(z, s); esc(:(@assert (size($z) == $s) string(summary($z),!=,$s))); end # for debugging

@size (macro with 1 method)

## Part -1. Types from the last project

Please copy the following types and related functions from the last project: `Vocab`,
`TextReader`, `Embed`, `Linear`, `mask!`.

In [2]:
# Your code here

# Vocab
struct Vocab
    w2i::Dict{String,Int}
    i2w::Vector{String}
    unk::Int
    eos::Int
    tokenizer
end

function Vocab(file::String; tokenizer=split, vocabsize=Inf, mincount=1, unk="<unk>", eos="<s>")
    cnt = Dict{String,Int}()
    for line in eachline(file)
        for word in tokenizer(line)
            cnt[word] = 1 + get(cnt, word, 0)
        end
    end
    delete!(cnt, unk); delete!(cnt, eos)
    words = sort!(collect(keys(cnt)); by=(w->cnt[w]), rev=true)
    if mincount > 1; i=findfirst(w -> cnt[w]<mincount, words); i!==nothing && (words=words[1:i-1]); end
    if unk != nothing; pushfirst!(words, unk); end
    if eos != nothing; pushfirst!(words, eos); end
    if length(words) > vocabsize; words = words[1:vocabsize]; end
    vocab = Dict(words[i] => i for i in 1:length(words))
    Vocab(vocab, words, get(vocab,unk,0), get(vocab,eos,0), tokenizer)
end

# Text Reader
struct TextReader
    file::String
    vocab::Vocab
end

Base.IteratorSize(::Type{TextReader}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{TextReader}) = Base.HasEltype()
Base.eltype(::Type{TextReader}) = Vector{Int}

function Base.iterate(r::TextReader, s=nothing)
    s === nothing && (s = open(r.file))
    if eof(s); close(s); return nothing; end
    v = r.vocab
    [ get(v.w2i, w, v.unk) for w in v.tokenizer(readline(s)) ], s
end

# Embedding Layer
struct Embed; w; end

function Embed(vocabsize::Int, embedsize::Int)
    # Your code here
    Embed(param(embedsize, vocabsize))
end

function (l::Embed)(x)
    # Your code here
    l.w[:,x]
end

# Linear Layer
struct Linear; w; b; end

function Linear(inputsize::Int, outputsize::Int)
    Linear(param(outputsize, inputsize), param0(outputsize))
end

function (l::Linear)(x)
    l.w * x .+ l.b
end

# Masking
function mask!(a,pad)
    # Your code here
    matrix = copy(a)
    for row in 1:size(matrix, 1)
        col = size(matrix, 2)
        while col > 1 && matrix[row, col] == pad 
            if matrix[row, col - 1] == pad; matrix[row, col] = 0; end
            col -= 1
        end
    end
    return matrix
end

mask! (generic function with 1 method)

## Part 0. Load data

We will use the Turkish-English pair from the [TED Talks Dataset](https://github.com/neulab/word-embeddings-for-nmt) for our experiments.

In [3]:
datadir = "datasets/tr_to_en"

if !isdir(datadir)
    download("http://www.phontron.com/data/qi18naacl-dataset.tar.gz", "qi18naacl-dataset.tar.gz")
    run(`tar xzf qi18naacl-dataset.tar.gz`)
end

if !isdefined(Main, :tr_vocab)
    tr_vocab = Vocab("$datadir/tr.train", mincount=5)
    en_vocab = Vocab("$datadir/en.train", mincount=5)
    tr_train = TextReader("$datadir/tr.train", tr_vocab)
    en_train = TextReader("$datadir/en.train", en_vocab)
    tr_dev = TextReader("$datadir/tr.dev", tr_vocab)
    en_dev = TextReader("$datadir/en.dev", en_vocab)
    tr_test = TextReader("$datadir/tr.test", tr_vocab)
    en_test = TextReader("$datadir/en.test", en_vocab)
    @info "Testing data"
    @test length(tr_vocab.i2w) == 38126
    @test length(first(tr_test)) == 16
    @test length(collect(tr_test)) == 5029
end

┌ Info: Testing data
└ @ Main In[3]:17


Test Passed
  Expression: length(collect(tr_test)) == 5029
   Evaluated: 5029 == 5029

## Part 1. Minibatching

For minibatching we are going to design a new iterator: `MTData`. This iterator is built
on top of two TextReaders `src` and `tgt` that produce parallel sentences for source and
target languages.

In [4]:
struct MTData
    src::TextReader        # reader for source language data
    tgt::TextReader        # reader for target language data
    batchsize::Int         # desired batch size
    maxlength::Int         # skip if source sentence above maxlength
    batchmajor::Bool       # batch dims (B,T) if batchmajor=false (default) or (T,B) if true.
    bucketwidth::Int       # batch sentences with length within bucketwidth of each other
    buckets::Vector        # sentences collected in separate arrays called buckets for each length range
    batchmaker::Function   # function that turns a bucket into a batch.
end

function MTData(src::TextReader, tgt::TextReader; batchmaker = arraybatch, batchsize = 128, maxlength = typemax(Int),
                batchmajor = false, bucketwidth = 10, numbuckets = min(128, maxlength ÷ bucketwidth))
    buckets = [ [] for i in 1:numbuckets ] # buckets[i] is an array of sentence pairs with similar source sentence length
    MTData(src, tgt, batchsize, maxlength, batchmajor, bucketwidth, buckets, batchmaker)
end

Base.IteratorSize(::Type{MTData}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{MTData}) = Base.HasEltype()
Base.eltype(::Type{MTData}) = NTuple{2}

### iterate(::MTData)

Define the `iterate` function for the `MTData` iterator. `iterate` should return a
`(batch, state)` pair or `nothing` if there are no more batches.  The `batch` is a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. The `state` is a pair of `(src_state,tgt_state)` which can be used
to iterate `d.src` and `d.tgt` to get more sentences.  `iterate(d)` without a second
argument should initialize `d` by emptying its buckets and calling `iterate` on the inner
iterators `d.src` and `d.tgt` without a state. Please review the documentation on
iterators from the last project.

To keep similar length sentences together `MTData` uses arrays of similar length sentence
pairs called buckets.  Specifically, the `(src_sentence,tgt_sentence)` pairs coming from
`src` and `tgt` are pushed into `d.buckets[i]` when the length of the source sentence is
in the range `((i-1)*d.bucketwidth+1):(i*d.bucketwidth)`. When one of the buckets reaches
`d.batchsize` `d.batchmaker` is called with the full bucket producing a 2-D batch, the
bucket is emptied and the batch is returned. If `src` and `tgt` are exhausted the
remaining partially full buckets are turned into batches and returned in any order. If the
source sentence length is larger than `length(d.buckets)*d.bucketwidth`, the last bucket
is used.

Sentences above a certain length can be skipped using the `d.maxlength` field, and
transposed `x,y` arrays can be produced using the `d.batchmajor` field.

In [5]:
function Base.iterate(d::MTData, state=nothing)
    # Your code here
     if state === nothing
        for b in d.buckets; empty!(b); end
    end
    bucket,ibucket = nothing,nothing
    while true
        if state === nothing
            iter1,iter2 = iterate(d.src),iterate(d.tgt)
        else
            iter1,iter2 = iterate(d.src, state[1]), iterate(d.tgt, state[2])
        end
        if iter1 === nothing || iter2 === nothing
            ibucket = findfirst(x -> !isempty(x), d.buckets)
            bucket = (ibucket === nothing ? nothing : d.buckets[ibucket])
            break
        else
            sent1, state1 = iter1; sent2, state2 = iter2
            state = (state1, state2)
            ## Filter and sort based on src sentence length
            if length(sent1) > d.maxlength || length(sent1) == 0; continue; end
            ibucket = min(1 + (length(sent1)-1) ÷ d.bucketwidth, length(d.buckets))
            bucket = d.buckets[ibucket]
            push!(bucket, (sent1, sent2))
            if length(bucket) === d.batchsize; break; end
        end
    end
    if bucket === nothing; return nothing; end
    batch = d.batchmaker(d, bucket)
    empty!(bucket)
    return batch, state
end

### arraybatch

Define `arraybatch(d, bucket)` to be used as the default `d.batchmaker`. `arraybatch`
takes an `MTData` object and an array of sentence pairs `bucket` and returns a
`(x::Matrix{Int},y::Matrix{Int})` pair where `x` is a `(batchsize,srclength)` batch of
source language sentences and `y` is a `(batchsize,tgtlength)` batch of parallel target
language translations. Note that the sentences in the bucket do not have any `eos` tokens
and they may have different lengths. `arraybatch` should copy the source sentences into
`x` padding shorter ones on the left with `eos` tokens. It should copy the target
sentences into `y` with an `eos` token in the beginning and end of each sentence and
shorter sentences padded on the right with extra `eos` tokens.

In [6]:
function arraybatch(d::MTData, bucket)
    
    batch_size = length(bucket)
    eos = d.src.vocab.eos
    
    source = [data[1] for data in bucket]; target = [data[2] for data in bucket]
    source_len = maximum(length.(source)); target_len = maximum(length.(target))
    
    sbatched = fill(eos, batch_size, source_len); tbatched = fill(eos, batch_size, target_len+2)
    for idx in 1:batch_size
        sbatched[idx, 1+end-length(source[idx]):end] = source[idx]
        tbatched[idx, 1] = eos; tbatched[idx, 2:1+length(target[idx])] = target[idx]; tbatched[idx, 2+length(target[idx])] = eos
    end
    if d.batchmajor == true; sbatched, tbatched = sbatched', tbatched'; end
    return sbatched, tbatched
    
end

arraybatch (generic function with 1 method)

In [7]:
@info "Testing MTData"
dtrn = MTData(tr_train, en_train)
ddev = MTData(tr_dev, en_dev)
dtst = MTData(tr_test, en_test)

x,y = first(dtst)
@test length(collect(dtst)) == 48
@test size.((x,y)) == ((128,10),(128,24))
@test x[1,1] == tr_vocab.eos
@test x[1,end] != tr_vocab.eos
@test y[1,1] == en_vocab.eos
@test y[1,2] != en_vocab.eos
@test y[1,end] == en_vocab.eos

┌ Info: Testing MTData
└ @ Main In[7]:1


Test Passed
  Expression: y[1, end] == en_vocab.eos
   Evaluated: 1 == 1

## Part 2. Sequence to sequence model without attention

In this part we will define a simple sequence to sequence encoder-decoder model for
machine translation.

In [8]:
struct S2S_v1
    srcembed::Embed     # source language embedding
    encoder::RNN        # encoder RNN (can be bidirectional)
    tgtembed::Embed     # target language embedding
    decoder::RNN        # decoder RNN
    projection::Linear  # converts decoder output to vocab scores
    dropout::Real       # dropout probability to prevent overfitting
    srcvocab::Vocab     # source language vocabulary
    tgtvocab::Vocab     # target language vocabulary
end

### S2S_v1 constructor

Define the S2S_v1 constructor using your predefined layer types (Embed, Linear), and the
Knet RNN type. Please review the RNN documentation using `@doc RNN`, paying attention to
the following options in particular: `numLayers`, `bidirectional`, `dropout`, `atype`.
The last one is important if you experiment with array types other than the
default `KnetArray{Float32}`: make sure the RNNs use the same array type as the other
layers. Note that if the encoder is bidirectional, its `numLayers` should be half of the
decoder so that their hidden states match in size.

In [9]:
function S2S_v1(hidden::Int,         # hidden size for both the encoder and decoder RNN
                srcembsz::Int,       # embedding size for source language
                tgtembsz::Int,       # embedding size for target language
                srcvocab::Vocab,     # vocabulary for source language
                tgtvocab::Vocab;     # vocabulary for target language
                layers=1,            # number of layers
                bidirectional=false, # whether encoder RNN is bidirectional
                dropout=0)           # dropout probability
    # Your code here
    srcembed = Embed(length(srcvocab.i2w), srcembsz)
    if bidirectional
        encoder = RNN(srcembsz, hidden, rnnType=:relu, numLayers=layers/2, dropout=dropout)
    else
        encoder = RNN(srcembsz, hidden, rnnType=:relu, numLayers=layers, dropout=dropout)
    end
    tgtembed = Embed(length(tgtvocab.i2w), tgtembsz)
    decoder = RNN(tgtembsz, hidden, rnnType=:relu, numLayers=layers, dropout=dropout)
    projection = Linear(hidden, length(tgtvocab.i2w))
    
    return S2S_v1(srcembed, encoder, tgtembed, decoder, projection, dropout, srcvocab, tgtvocab)
end

S2S_v1

### S2S_v1 loss function

Define the S2S_v1 loss function that takes `src`, a source language minibatch, and `tgt`,
a target language minibatch and returns either a `(total_loss, num_words)` pair if
`average=false`, or `(total_loss/num_words)` average if `average=true`.

Assume that `src` and `tgt` are integer arrays of size `(B,Tx)` and `(B,Ty)` respectively,
where `B` is the batch size, `Tx` is the length of the longest source sequence, `Ty` is
the length of the longest target sequence. The `src` sequences only contain words, the
`tgt` sequences surround the words with `eos` tokens at the start and end. This allows
columns `tgt[:,1:end-1]` to be used as the decoder input and `tgt[:,2:end]` as the desired
decoder output.

Assume any shorter sentences in the batches have been padded with extra `eos` tokens on
the left for `src` and on the right for `tgt`. Don't worry about masking `src` for the
encoder, it doesn't have a significant effect on the loss. However do mask `tgt` before
`nll`: you do not want the padding tokens to be counted in the loss calculation.

Please review `@doc RNN`: in particular the `r.c` and `r.h` fields can be used to get/set
the cell and hidden arrays of an RNN (note that `0` and `nothing` act as special values).

RNNs take a dropout value at construction and apply dropout to the input of every layer if
it is non-zero. You need to handle dropout for other layers in the loss function or in
layer definitions as necessary.

In [12]:
function (s::S2S_v1)(src, tgt; average=true)
    # Your code here
    s.encoder.h, s.encoder.c = 0, 0
    s.decoder.h, s.decoder.c = 0, 0
    #B, Tx, Ty = size(src, 1), size(src, 2), size(tgt, 2)
    tgt_decoder_input  = tgt[:,1:end-1]
    tgt_decoder_output = tgt[:,2:end]
    println("Decoder out: $(size(tgt_decoder_output))")
    srcembed = s.srcembed(src)
    rnn_encoder_out = s.encoder(srcembed)
    println("Encoder: $(size(rnn_encoder_out))")
    tgtembed = s.tgtembed(tgt_decoder_input)
    rnn_decoder_out = s.decoder(rnn_encoder_out)
    println("Decoder: $(size(rnn_decoder_out))")
    H, B, T = size(rnn_decoder_out, 1), size(rnn_decoder_out, 2), size(rnn_decoder_out, 3)
    output = s.projection(dropout(reshape(rnn_decoder_out, H, B * T), s.dropout))
    println("Output: $(size(output))")
    scores = reshape(output, size(output, 1), B, T)
    println("Scores: $(size(scores))")
    return nll(scores, mask!(tgt_decoder_output, s.tgtvocab.eos); dims=1, average=average)
    
    
end

In [13]:
@info "Testing S2S_v1"
Random.seed!(1)
model = S2S_v1(512, 512, 512, tr_vocab, en_vocab; layers=2, bidirectional=true, dropout=0.2)
(x,y) = first(dtst)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
@test model(x,y; average=false) == (14096.252f0, 1432)

Decoder out: (128, 23)
Encoder: (512, 128, 10)
Decoder: (512, 128, 10)
Output: (18857, 1280)
Scores: (18857, 128, 10)
Error During Test at In[13]:7
  Test threw exception
  Expression: model(x, y; average = false) == (14096.252f0, 1432)
  DimensionMismatch("")
  Stacktrace:
    [1] findindices(scores::KnetArray{Float32, 3}, labels::Matrix{Int64}; dims::Int64)
      @ Knet.Ops20 ~/.julia/packages/Knet/YIFWC/src/ops20/loss.jl:123
    [2] nll(scores::KnetArray{Float32, 3}, labels::Matrix{Int64}; dims::Int64, average::Bool)
      @ Knet.Ops20 ~/.julia/packages/Knet/YIFWC/src/ops20/loss.jl:38
    [3] (::S2S_v1)(src::Matrix{Int64}, tgt::Matrix{Int64}; average::Bool)
      @ Main ./In[12]:20
    [4] top-level scope
      @ /opt/julia-1.7.2/share/julia/stdlib/v1.7/Test/src/Test.jl:445
    [5] eval
      @ ./boot.jl:373 [inlined]
    [6] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base ./loading.jl:1196
    [7] softscope_include_string(m:

┌ Info: Testing S2S_v1
└ @ Main In[13]:1


LoadError: [91mThere was an error during testing[39m

### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(x,y)` pairs such as `MTData` and `model(x,y;average)` is a model like
`S2S_v1` that computes loss on a single `(x,y)` pair.

In [ ]:
function loss(model, data; average=true)
    # Your code here
    loss = mean([model(x,y) for (x,y) in data])
    return loss
end

In [ ]:
@info "Testing loss"
@test loss(model, dtst, average=false) == (1.0427646f6, 105937)
# Your loss can be slightly different due to different ordering of words in the vocabulary.
# The reference vocabulary starts with eos, unk, followed by words in decreasing frequency.
# Also, because we do not mask src, different batch sizes may lead to slightly different
# losses. The test above gives (1.0430301f6, 105937) with batchsize==1.

### Training SGD_v1

The following function can be used to train our model. `trn` is the training data, `dev`
is used to determine the best model, `tst...` can be zero or more small test datasets for
loss reporting. It returns the model that does best on `dev`.

In [ ]:
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=100) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

You should be able to get under 3.40 dev loss with the following settings in 10
epochs. The training speed on a V100 is about 3 mins/epoch or 40K words/sec, K80 is about
6 times slower. Using settings closer to the Luong paper (per-sentence loss rather than
per-word loss, SGD with lr=1, gclip=1 instead of Adam), you can get to 3.17 dev loss in
about 25 epochs. Using dropout and shuffling batches before each epoch significantly
improve the dev loss. You can play around with hyperparameters but I doubt results will
get much better without attention. To verify your training, here is the dev loss I
observed at the beginning of each epoch in one training session:
`[9.83, 4.60, 3.98, 3.69, 3.52, 3.41, 3.35, 3.32, 3.30, 3.31, 3.33]`

In [ ]:
@info "Training S2S_v1"
epochs = 10
ctrn = collect(dtrn)
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trn20 = ctrn[1:20]
dev38 = collect(ddev)
# Uncomment this to train the model (This takes about 30 mins on a V100, 60 mins on a T4):
# model = train!(model, trnx10, dev38, trn20)
# Uncomment this to save the model:
# Knet.save("s2s_v1.jld2","model",model)
# Uncomment this to load the model:
# model = Knet.load("s2s_v1.jld2","model")

### Generating translations

With a single argument, a `S2S_v1` object should take it as a batch of source sentences
and generate translations for them. After passing `src` through the encoder and copying
its hidden states to the decoder, the decoder is run starting with an initial input of all
`eos` tokens. Highest scoring tokens are appended to the output and used as input for the
subsequent decoder steps.  The decoder should stop generating when all sequences in the
batch have generated `eos` or when `stopfactor * size(src,2)` decoder steps are reached. A
correctly shaped target language batch should be returned.

In [ ]:
function (s::S2S_v1)(src::Matrix{Int}; stopfactor = 3)
    # Your code here
end

In [ ]:
# Utility to convert int arrays to sentence strings
function int2str(y,vocab)
    y = vec(y)
    ysos = findnext(w->!isequal(w,vocab.eos), y, 1)
    ysos == nothing && return ""
    yeos = something(findnext(isequal(vocab.eos), y, ysos), 1+length(y))
    join(vocab.i2w[y[ysos:yeos-1]], " ")
end

In [ ]:
# Uncomment and run these lines if you get a "CUDNNError: CUDNN_STATUS_INTERNAL_ERROR (code 4)" error from the cell below.
# Knet.save("s2s_v1.jld2","model",model)
# model = Knet.load("s2s_v1.jld2","model")

In [ ]:
@info "Generating some translations"
d = MTData(tr_dev, en_dev, batchsize=1) |> collect
(src,tgt) = rand(d)
out = model(src)
println("SRC: ", int2str(src,model.srcvocab))
println("REF: ", int2str(tgt,model.tgtvocab))
println("OUT: ", int2str(out,model.tgtvocab))
# Here is a sample output:
# SRC: çin'e 15 şubat 2006'da ulaştım .
# REF: i made it to china on february 15 , 2006 .
# OUT: i got to china , china , at the last 15 years .

### Calculating BLEU

BLEU is the most commonly used metric to measure translation quality. The following should
take a model and some data, generate translations and calculate BLEU.

In [ ]:
function bleu(s2s,d::MTData)
    d = MTData(d.src,d.tgt,batchsize=1)
    reffile = d.tgt.file
    hypfile,hyp = mktemp()
    for (x,y) in progress(collect(d))
        g = s2s(x)
        for i in 1:size(y,1)
            println(hyp, int2str(g[i,:], d.tgt.vocab))
        end
    end
    close(hyp)
    isfile("multi-bleu.perl") || download("https://github.com/moses-smt/mosesdecoder/raw/master/scripts/generic/multi-bleu.perl", "multi-bleu.perl")
    run(pipeline(`cat $hypfile`,`perl multi-bleu.perl $reffile`))
    return hypfile
end

Calculating dev BLEU takes about 100 secs on a T4. We get about 8.0 BLEU which is pretty
low. As can be seen from the sample translations a loss of ~3+ (perplexity ~20+) or a BLEU
of ~8 is not sufficient to generate meaningful translations.

In [ ]:
@info "Calculating BLEU"
bleu(model, ddev)

To improve the quality of translations we can use more training data, different training
and model parameters, or preprocess the input/output: e.g. splitting Turkish words to make
suffixes look more like English function words may help. Other architectures,
e.g. attention and transformer, perform significantly better than this simple S2S model.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*